## Farmer Market Data Cleanup Steps

In [332]:
import pandas as pd
import folium
import sqlite3
import re

In [265]:
df = pd.read_csv('~/Downloads/farmersmarkets-2017-01-10.csv')
df.dtypes

FMID               int64
MarketName        object
Website           object
Facebook          object
Twitter           object
Youtube           object
OtherMedia        object
street            object
city              object
County            object
State             object
zip               object
Season1Date       object
Season1Time       object
Season2Date       object
Season2Time       object
Season3Date       object
Season3Time       object
Season4Date       object
Season4Time       object
x                float64
y                float64
Location          object
Credit            object
WIC               object
WICcash           object
SFMNP             object
SNAP              object
Organic           object
Bakedgoods        object
Cheese            object
Crafts            object
Flowers           object
Eggs              object
Seafood           object
Herbs             object
Vegetables        object
Honey             object
Jams              object
Maple             object


## Data Analysis and cleaning using Pandas

In [266]:
df.isna().sum()

FMID                0
MarketName          0
Website          3458
Facebook         4869
Twitter          7668
Youtube          8504
OtherMedia       8027
street            285
city               40
County            538
State               0
zip               944
Season1Date      3279
Season1Time      3140
Season2Date      8236
Season2Time      8251
Season3Date      8586
Season3Time      8590
Season4Date      8658
Season4Time      8658
x                  29
y                  29
Location         5729
Credit              0
WIC                 0
WICcash             0
SFMNP               0
SNAP                0
Organic             0
Bakedgoods       3023
Cheese           3023
Crafts           3023
Flowers          3023
Eggs             3023
Seafood          3023
Herbs            3023
Vegetables       3023
Honey            3023
Jams             3023
Maple            3023
Meat             3023
Nursery          3023
Nuts             3023
Plants           3023
Poultry          3023
Prepared  

## Update Time cleanup
1990
1990/01/01
1990/mar/01
Emoty to NaN

In [322]:
df['updateTime'] = df['updateTime'].apply(lambda col: pd.to_datetime(col, errors='ignore'))
df.updateTime.head()

0   2016-06-28 12:10:09
1   2016-04-09 20:05:17
2   2016-07-15 19:20:33
3   2013-01-01 00:00:00
4   2014-10-28 09:49:46
Name: updateTime, dtype: datetime64[ns]

In [268]:
#Change column datatypes to boolean
df[df.columns[23:-1]]=df[df.columns[23:-1]].astype("boolean")

TypeError: Need to pass bool-like values

In [269]:
# Check number of columns have missing values
df.isna().sum()

FMID                0
MarketName          0
Website          3458
Facebook         4869
Twitter          7668
Youtube          8504
OtherMedia       8027
street            285
city               40
County            538
State               0
zip               944
Season1Date      3279
Season1Time      3140
Season2Date      8236
Season2Time      8251
Season3Date      8586
Season3Time      8590
Season4Date      8658
Season4Time      8658
x                  29
y                  29
Location         5729
Credit              0
WIC                 0
WICcash             0
SFMNP               0
SNAP                0
Organic             0
Bakedgoods       3023
Cheese           3023
Crafts           3023
Flowers          3023
Eggs             3023
Seafood          3023
Herbs            3023
Vegetables       3023
Honey            3023
Jams             3023
Maple            3023
Meat             3023
Nursery          3023
Nuts             3023
Plants           3023
Poultry          3023
Prepared  

## Exclude missing lat & lon from Data

In [328]:
df=df[df['x'].notna()]
df=df[df['y'].notna()]

In [329]:
## Check for duplicate in market id

In [330]:
df['FMID'].value_counts(ascending=False).head()

1012063    1
1001862    1
1007858    1
1002692    1
1002833    1
Name: FMID, dtype: int64

## Plot market data in USA map- no data cleaning is required (U0)

In [240]:
# Create a map of U.S. farmers markets using latitude and longitude values
map_markets = folium.Map(location=[39.8283, -98.5795], zoom_start=5, popup='Portland, OR')

# add markers to map
for  lng,lat, market, state in zip(df['x'],df['y'], df['MarketName'], df['State']):
    label = '{}, {}'.format(market, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#31cc34',
        fill_opacity=0.5,
        parse_html=False).add_to(map_markets)

map_markets

ValueError: Location values cannot contain NaNs.

## Export table to SQL Lite For data normalization

In [241]:
#Examport SQL data to Table
conn = sqlite3.connect("farmers_market.db")

In [ ]:
df.to_sql("farmers_market", conn, if_exists="replace")

In [11]:
conn.execute(
    """
    create table farmers_market as
    select * from tmp
    """)

Validate It

In [ ]:
conn.cursor().execute(
    """
    select count(*) from farmers_market
    """).fetchall()

## Generate Has Social media grouped columns

In [273]:
df['has_social_media']=df.apply(lambda x:  pd.isna(x.Facebook) is False or pd.isna(x.Twitter) is False or  pd.isna(x.Youtube) is False or pd.isna(x.Website) is False or pd.isna(x.OtherMedia) is False,axis=1)

In [274]:
df.has_social_media.value_counts()

True     6029
False    2607
Name: has_social_media, dtype: int64

## Address Cleaning Rules

In [297]:
fm_subset=df[["FMID",'MarketName',"Facebook","Twitter","Youtube","Website",'has_social_media',"city","State","street","County","x","y",'Credit','WIC','WICcash','SFMNP','Organic','Flowers','Vegetables','Meat','Nursery','Wine','Coffee','Fruits','PetFood','WildHarvested','updateTime']]

In [298]:
import pandas_usaddress,re

In [299]:
fm_subset=pandas_usaddress.tag(fm_subset, ['street'], granularity='medium', standardize=False)

/var/folders/qd/tg9lj7vj279dq94pqm2rj3n80000gn/T/ipykernel_56493/3169073722.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  fm_subset=pandas_usaddress.tag(fm_subset, ['street'], granularity='medium', standardize=False)


In [300]:
fm_subset.columns

Index(['FMID', 'MarketName', 'Facebook', 'Twitter', 'Youtube', 'Website',
       'has_social_media', 'city', 'State', 'street', 'County', 'x', 'y',
       'Credit', 'WIC', 'WICcash', 'SFMNP', 'Organic', 'Flowers', 'Vegetables',
       'Meat', 'Nursery', 'Wine', 'Coffee', 'Fruits', 'PetFood',
       'WildHarvested', 'updateTime', 'AddressNumber', 'PlaceName',
       'StateName', 'StreetName', 'StreetNamePreDirectional',
       'StreetNamePostDirectional', 'ZipCode', 'StreetNamePrefix',
       'StreetNameSuffix', 'USPSBox', 'OccupancySuite'],
      dtype='object')

In [301]:
def address_number_sumamry(fm_subset):
    print(fm_subset['AddressNumber'].apply(lambda x: re.search('[^\d]+',str(x)) is None).sum())

## AddressNumber cleanup
### Before update AddressNumber

In [280]:
address_number_sumamry(fm_subset)

4913


In [302]:
def address_number_fun(strr):
    if pd.isna(strr) :
        return strr
    p=re.compile('(\d+)?[^0-9]*')
    m=p.match(strr)
    grps = m.groups()
    return grps[0] if len(grps)>0 else pd.NaT

In [282]:
fm_subset['AddressNumber']=fm_subset['AddressNumber'].apply(address_number_fun)

### After Cleanup

In [283]:
address_number_sumamry(fm_subset)

5195


## Street Suffx Cleanup

#Prepare street suffix
str_sfx=pd.read_csv('dataset/street_suffix_abbvr.csv')
str_sfx.fillna(method='ffill',inplace=True)
str_sfx = str_sfx.applymap(lambda s: s.lower() if type(s) == str else s)
str_sfx.set_index('abbvr',inplace=True).to_dict()

In [303]:
str_sfx={'allee': 'aly','alley': 'aly','ally': 'aly','aly': 'aly','anex': 'anx','annex': 'anx','annx': 'anx','anx': 'anx','arc': 'arc','arcade': 'arc','av': 'ave','ave': 'ave','aven': 'ave','avenu': 'ave','avenue': 'ave','avn': 'ave','avnue': 'ave','bayoo': 'byu','bayou': 'byu','bch': 'bch','beach': 'bch','bend': 'bnd','bnd': 'bnd','blf': 'blf','bluf': 'blf','bluff': 'blf','bluffs': 'blfs','bot': 'btm','btm': 'btm','bottm': 'btm','bottom': 'btm','blvd': 'blvd','boul': 'blvd','boulevard': 'blvd','boulv': 'blvd','br': 'br','brnch': 'br','branch': 'br','brdge': 'brg','brg': 'brg','bridge': 'brg','brk': 'brk','brook': 'brk','brooks': 'brks','burg': 'bg','burgs': 'bgs','byp': 'byp','bypa': 'byp','bypas': 'byp','bypass': 'byp','byps': 'byp','camp': 'cp','cp': 'cp','cmp': 'cp','canyn': 'cyn','canyon': 'cyn','cnyn': 'cyn','cape': 'cpe','cpe': 'cpe','causeway': 'cswy','causwa': 'cswy','cswy': 'cswy','cen': 'ctr','cent': 'ctr','center': 'ctr','centr': 'ctr','centre': 'ctr','cnter': 'ctr','cntr': 'ctr','ctr': 'ctr','centers': 'ctrs','cir': 'cir','circ': 'cir','circl': 'cir','circle': 'cir','crcl': 'cir','crcle': 'cir','circles': 'cirs','clf': 'clf','cliff': 'clf','clfs': 'clfs','cliffs': 'clfs','clb': 'clb','club': 'clb','common': 'cmn','commons': 'cmns','cor': 'cor','corner': 'cor','corners': 'cors','cors': 'cors','course': 'crse','crse': 'crse','court': 'ct','ct': 'ct','courts': 'cts','cts': 'cts','cove': 'cv','cv': 'cv','coves': 'cvs','creek': 'crk','crk': 'crk','crescent': 'cres','cres': 'cres','crsent': 'cres','crsnt': 'cres','crest': 'crst','crossing': 'xing','crssng': 'xing','xing': 'xing','crossroad': 'xrd','crossroads': 'xrds','curve': 'curv','dale': 'dl','dl': 'dl','dam': 'dm','dm': 'dm','div': 'dv','divide': 'dv','dv': 'dv','dvd': 'dv','dr': 'dr','driv': 'dr','drive': 'dr','drv': 'dr','drives': 'drs','est': 'est','estate': 'est','estates': 'ests','ests': 'ests','exp': 'expy','expr': 'expy','express': 'expy','expressway': 'expy','expw': 'expy','expy': 'expy','ext': 'ext','extension': 'ext','extn': 'ext','extnsn': 'ext','exts': 'exts','fall': 'fall','falls': 'fls','fls': 'fls','ferry': 'fry','frry': 'fry','fry': 'fry','field': 'fld','fld': 'fld','fields': 'flds','flds': 'flds','flat': 'flt','flt': 'flt','flats': 'flts','flts': 'flts','ford': 'frd','frd': 'frd','fords': 'frds','forest': 'frst','forests': 'frst','frst': 'frst','forg': 'frg','forge': 'frg','frg': 'frg','forges': 'frgs','fork': 'frk','frk': 'frk','forks': 'frks','frks': 'frks','fort': 'ft','frt': 'ft','ft': 'ft','freeway': 'fwy','freewy': 'fwy','frway': 'fwy','frwy': 'fwy','fwy': 'fwy','garden': 'gdn','gardn': 'gdn','grden': 'gdn','grdn': 'gdn','gardens': 'gdns','gdns': 'gdns','grdns': 'gdns','gateway': 'gtwy','gatewy': 'gtwy','gatway': 'gtwy','gtway': 'gtwy','gtwy': 'gtwy','glen': 'gln','gln': 'gln','glens': 'glns','green': 'grn','grn': 'grn','greens': 'grns','grov': 'grv','grove': 'grv','grv': 'grv','groves': 'grvs','harb': 'hbr','harbor': 'hbr','harbr': 'hbr','hbr': 'hbr','hrbor': 'hbr','harbors': 'hbrs','haven': 'hvn','hvn': 'hvn','ht': 'hts','hts': 'hts','highway': 'hwy','highwy': 'hwy','hiway': 'hwy','hiwy': 'hwy','hway': 'hwy','hwy': 'hwy','hill': 'hl','hl': 'hl','hills': 'hls','hls': 'hls','hllw': 'holw','hollow': 'holw','hollows': 'holw','holw': 'holw','holws': 'holw','inlt': 'inlt','is': 'is','island': 'is','islnd': 'is','islands': 'iss','islnds': 'iss','iss': 'iss','isle': 'isle','isles': 'isle','jct': 'jct','jction': 'jct','jctn': 'jct','junction': 'jct','junctn': 'jct','juncton': 'jct','jctns': 'jcts','jcts': 'jcts','junctions': 'jcts','key': 'ky','ky': 'ky','keys': 'kys','kys': 'kys','knl': 'knl','knol': 'knl','knoll': 'knl','knls': 'knls','knolls': 'knls','lk': 'lk','lake': 'lk','lks': 'lks','lakes': 'lks','land': 'land','landing': 'lndg','lndg': 'lndg','lndng': 'lndg','lane': 'ln','ln': 'ln','lgt': 'lgt','light': 'lgt','lights': 'lgts','lf': 'lf','loaf': 'lf','lck': 'lck','lock': 'lck','lcks': 'lcks','locks': 'lcks','ldg': 'ldg','ldge': 'ldg','lodg': 'ldg','lodge': 'ldg','loop': 'loop','loops': 'loop','mall': 'mall','mnr': 'mnr','manor': 'mnr','manors': 'mnrs','mnrs': 'mnrs','meadow': 'mdw','mdw': 'mdws','mdws': 'mdws','meadows': 'mdws','medows': 'mdws','mews': 'mews','mill': 'ml','mills': 'mls','missn': 'msn','mssn': 'msn','motorway': 'mtwy','mnt': 'mt','mt': 'mt','mount': 'mt','mntain': 'mtn','mntn': 'mtn','mountain': 'mtn','mountin': 'mtn','mtin': 'mtn','mtn': 'mtn','mntns': 'mtns','mountains': 'mtns','nck': 'nck','neck': 'nck','orch': 'orch','orchard': 'orch','orchrd': 'orch','oval': 'oval','ovl': 'oval','overpass': 'opas','park': 'park','prk': 'park','parks': 'park','parkway': 'pkwy','parkwy': 'pkwy','pkway': 'pkwy','pkwy': 'pkwy','pky': 'pkwy','parkways': 'pkwy','pkwys': 'pkwy','pass': 'pass','passage': 'psge','path': 'path','paths': 'path','pike': 'pike','pikes': 'pike','pine': 'pne','pines': 'pnes','pnes': 'pnes','pl': 'pl','plain': 'pln','pln': 'pln','plains': 'plns','plns': 'plns','plaza': 'plz','plz': 'plz','plza': 'plz','point': 'pt','pt': 'pt','points': 'pts','pts': 'pts','port': 'prt','prt': 'prt','ports': 'prts','prts': 'prts','pr': 'pr','prairie': 'pr','prr': 'pr','rad': 'radl','radial': 'radl','radiel': 'radl','radl': 'radl','ramp': 'ramp','ranch': 'rnch','ranches': 'rnch','rnch': 'rnch','rnchs': 'rnch','rapid': 'rpd','rpd': 'rpd','rapids': 'rpds','rpds': 'rpds','rest': 'rst','rst': 'rst','rdg': 'rdg','rdge': 'rdg','ridge': 'rdg','rdgs': 'rdgs','ridges': 'rdgs','riv': 'riv','river': 'riv','rvr': 'riv','rivr': 'riv','rd': 'rd','road': 'rd','roads': 'rds','rds': 'rds','route': 'rte','row': 'row','rue': 'rue','run': 'run','shl': 'shl','shoal': 'shl','shls': 'shls','shoals': 'shls','shoar': 'shr','shore': 'shr','shr': 'shr','shoars': 'shrs','shores': 'shrs','shrs': 'shrs','skyway': 'skwy','spg': 'spg','spng': 'spg','spring': 'spg','sprng': 'spg','spgs': 'spgs','spngs': 'spgs','springs': 'spgs','sprngs': 'spgs','spur': 'spur','spurs': 'spur','sq': 'sq','sqr': 'sq','sqre': 'sq','squ': 'sq','square': 'sq','sqrs': 'sqs','squares': 'sqs','sta': 'sta','station': 'sta','statn': 'sta','stn': 'sta','stra': 'stra','strav': 'stra','straven': 'stra','stravenue': 'stra','stravn': 'stra','strvn': 'stra','strvnue': 'stra','stream': 'strm','streme': 'strm','strm': 'strm','street': 'st','strt': 'st','st': 'st','str': 'st','streets': 'sts','smt': 'smt','sumit': 'smt','sumitt': 'smt','summit': 'smt','ter': 'ter','terr': 'ter','terrace': 'ter','throughway': 'trwy','trace': 'trce','traces': 'trce','trce': 'trce','track': 'trak','tracks': 'trak','trak': 'trak','trk': 'trak','trks': 'trak','trafficway': 'trfy','trail': 'trl','trails': 'trl','trl': 'trl','trls': 'trl','trailer': 'trlr','trlr': 'trlr','trlrs': 'trlr','tunel': 'tunl','tunl': 'tunl','tunls': 'tunl','tunnel': 'tunl','tunnels': 'tunl','tunnl': 'tunl','trnpk': 'tpke','turnpike': 'tpke','turnpk': 'tpke','underpass': 'upas','un': 'un','union': 'un','unions': 'uns','valley': 'vly','vally': 'vly','vlly': 'vly','vly': 'vly','valleys': 'vlys','vlys': 'vlys','vdct': 'via','via': 'via','viadct': 'via','viaduct': 'via','view': 'vw','vw': 'vw','views': 'vws','vws': 'vws','vill': 'vlg','villag': 'vlg','village': 'vlg','villg': 'vlg','villiage': 'vlg','vlg': 'vlg','villages': 'vlgs','vlgs': 'vlgs','ville': 'vl','vl': 'vl','vis': 'vis','vist': 'vis','vista': 'vis','vst': 'vis','vsta': 'vis','walk': 'walk','walks': 'walk','wall': 'wall','wy': 'way','way': 'way','ways': 'ways','well': 'wl','wells': 'wls','wls': 'wls'}

### Before suffix cleanup

In [285]:
fm_subset.StreetNameSuffix.value_counts()

street           1792
st               1013
ave               627
road              568
avenue            436
                 ... 
trl                 1
lodge               1
drives              1
river               1
river highway       1
Name: StreetNameSuffix, Length: 94, dtype: int64

In [304]:
fm_subset.replace({'StreetNameSuffix':str_sfx},inplace=True)

### After Cleanup

In [287]:
fm_subset['StreetNameSuffix'].value_counts()

st               2807
ave              1068
rd                906
dr                321
blvd              257
                 ... 
at                  1
ldg                 1
drs                 1
riv                 1
river highway       1
Name: StreetNameSuffix, Length: 74, dtype: int64

## Street Name direction
### Before Cleanup

In [305]:
fm_subset.StreetNamePreDirectional.value_counts()

n            310
s            292
e            259
w            258
south        195
east         174
north        172
west         171
ne            35
sw            33
se            18
nw            15
so             3
ave            2
southeast      2
northeast      2
northwest      2
to             1
southwest      1
of             1
Name: StreetNamePreDirectional, dtype: int64

In [289]:
dir_suffix={'n':"north",'s':'south','e':'east','w':'west','ne':'northeast','se':'southeast','nw':"northwest",'sw':'southwest'}

In [307]:
fm_subset.replace({'StreetNamePreDirectional':dir_suffix},inplace=True)

### After Cleanup

In [306]:
fm_subset.StreetNamePreDirectional.value_counts()

n            310
s            292
e            259
w            258
south        195
east         174
north        172
west         171
ne            35
sw            33
se            18
nw            15
so             3
ave            2
southeast      2
northeast      2
northwest      2
to             1
southwest      1
of             1
Name: StreetNamePreDirectional, dtype: int64

## Join Columns and remove multiple spaces

In [308]:
fm_subset['street']=fm_subset[['AddressNumber',
       'PlaceName', 'StateName', 'StreetName', 'StreetNamePreDirectional',
       'StreetNamePostDirectional', 'ZipCode', 'StreetNamePrefix',
       'StreetNameSuffix', 'USPSBox', 'OccupancySuite']].fillna('').astype(str).apply(lambda x: re.sub('\s+',' ',' '.join(x)).strip(), axis=1)

In [309]:
fm_subset.drop(['AddressNumber',
       'PlaceName', 'StateName', 'StreetName', 'StreetNamePreDirectional',
       'StreetNamePostDirectional', 'ZipCode', 'StreetNamePrefix',
       'StreetNameSuffix', 'USPSBox', 'OccupancySuite'],axis=1,inplace=True)

In [294]:
fm_subset.head()

,FMID,Facebook,Twitter,Youtube,Website,has_social_media,city,State,street,County,...,Flowers,Vegetables,Meat,Nursery,Wine,Coffee,Fruits,PetFood,WildHarvested,updateTime
0,1012063,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,https://sites.google.com/site/caledoniafarmers...,True,Danville,Vermont,,Caledonia,...,Y,Y,Y,N,N,Y,Y,Y,N,2016-06-28 12:10:09
1,1011871,NaN,NaN,NaN,http://Stearnshomestead.com,True,Parma,Ohio,6975 ridge rd,Cuyahoga,...,Y,Y,Y,N,N,N,Y,Y,N,2016-04-09 20:05:17
2,1011878,https://www.facebook.com/100MileMarket/?fref=ts,NaN,NaN,http://www.pfcmarkets.com,True,Kalamazoo,Michigan,507 harrison st,Kalamazoo,...,Y,Y,Y,N,N,N,Y,N,N,2016-07-15 19:20:33
3,1009364,NaN,NaN,NaN,http://thetownofsixmile.wordpress.com/,True,Six Mile,South Carolina,106 main south st,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-01-01 00:00:00
4,1010691,NaN,NaN,NaN,NaN,True,Lamar,Missouri,10th st,Barton,...,N,Y,Y,N,N,N,Y,N,N,2014-10-28 09:49:46


## Market Name Cleanup
### Cleanup alpha Numeric Texts

In [316]:
fm_subset['MarketName'] = fm_subset['MarketName'].apply(lambda x: re.sub('[^a-zA-Z\d\s:]', '', x))

### Remove Farmers Market from Name if Name is already more than 3 words

In [ ]:
def market_name_subs(fm):
    words = fm.split('\s')
    if len(words ) >3:
        return re.sub("((Farm)|Farmer)|(market)",'',fm,re.IGNORECASE)
    else:
        return fm


In [311]:
fm_subset.to_csv('dataset/fm_0716.csv')

In [312]:
fm_subset.columns

Index(['FMID', 'MarketName', 'Facebook', 'Twitter', 'Youtube', 'Website',
       'has_social_media', 'city', 'State', 'street', 'County', 'x', 'y',
       'Credit', 'WIC', 'WICcash', 'SFMNP', 'Organic', 'Flowers', 'Vegetables',
       'Meat', 'Nursery', 'Wine', 'Coffee', 'Fruits', 'PetFood',
       'WildHarvested', 'updateTime'],
      dtype='object')

In [331]:
fm_subset['MarketName'].head()

0     Caledonia Farmers Market Association  Danville
1                   Stearns Homestead Farmers Market
2                                    100 Mile Market
3                   106 S Main Street Farmers Market
4                10th Steet Community Farmers Market
Name: MarketName, dtype: object